# Boilerplate and training


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 50)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50,50)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(50,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = self.fc(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 59604959.91it/s]


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 2139450.62it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw


...

100%|██████████| 1648877/1648877 [00:00<00:00, 14716917.05it/s]


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 8370179.60it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=50, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=50, out_features=50, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [ ]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')


In [ ]:
# model = nn.Sequential(Normalize(), Net())
# model = model.to(device)
# model.train()

# train_model(model, num_epochs=10)
# torch.save(model.state_dict(), '/content/drive/MyDrive/CS521/weights_HW2.pt')

In [ ]:
def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

# Main part

In [ ]:
model = nn.Sequential(Normalize(), Net())
model.load_state_dict(torch.load('/content/drive/MyDrive/CS521/weights_HW2.pt'))

<ipython-input-6-5174b9c87ff7>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/CS521/weights_HW2.pt'))


<All keys matched successfully>

In [ ]:
test_model(model)

Accuracy on images: 95.86


In [ ]:
def IBP_bounds(model, x0, eps):
    model.eval()

    normalize = model[0]
    lower = normalize(x0) - eps
    upper = normalize(x0) + eps

    net = model[1]
    for layer in net.children():
        if isinstance(layer, nn.Linear):
            W = layer.weight.data
            b = layer.bias.data
            pos_W = torch.clamp(W, min=0)
            neg_W = torch.clamp(W, max=0)
            lower2 = neg_W @ upper + pos_W @ lower + b
            upper2 = pos_W @ upper + neg_W @ lower + b
            lower, upper = lower2, upper2
        elif isinstance(layer, nn.ReLU):
            lower = torch.clamp(lower, min=0)
            upper = torch.clamp(upper, min=0)
    return (lower, upper)

In [ ]:
k = 1000
for eps in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]:
    succ = 0
    for i in range(k):
        random_image, random_label = train_dataset[np.random.randint(0, len(train_dataset))]
        if model(random_image).argmax() != random_label:
            continue
        # plt.imshow(random_image.squeeze(), cmap='gray')
        # plt.show()
        random_image = random_image.flatten()
        original_logits = model(random_image)
        with torch.no_grad():
            lower, upper = IBP_bounds(model, random_image, eps)
            assert ((lower - 1e-6) <= original_logits).all() and ((upper + 1e-6) >= original_logits).all()
        succ += 1 if lower.argmax() == random_label else 0

    print(f'Success rate for eps={eps}: {succ/k}')

Success rate for eps=0.01: 0.962
Success rate for eps=0.02: 0.946
Success rate for eps=0.03: 0.928
Success rate for eps=0.04: 0.874
Success rate for eps=0.05: 0.794
Success rate for eps=0.06: 0.661
Success rate for eps=0.07: 0.588
Success rate for eps=0.08: 0.487
Success rate for eps=0.09: 0.425
Success rate for eps=0.1: 0.376


In [ ]:
k = 1000
for eps in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]:
    succ = 0
    for i in range(k):
        random_image, random_label = test_dataset[np.random.randint(0, len(test_dataset))]
        if model(random_image).argmax() != random_label:
            continue
        # plt.imshow(random_image.squeeze(), cmap='gray')
        # plt.show()
        random_image = random_image.flatten()
        correct = model(random_image).argmax()
        original_logits = model(random_image)
        with torch.no_grad():
            lower, upper = IBP_bounds(model, random_image, eps)
            assert ((lower - 1e-6) <= original_logits).all() and ((upper + 1e-6) >= original_logits).all()
        succ += 1 if lower.argmax() == correct else 0

    print(f'Success rate for eps={eps}: {succ/k}')

Success rate for eps=0.01: 0.965
Success rate for eps=0.02: 0.947
Success rate for eps=0.03: 0.921
Success rate for eps=0.04: 0.867
Success rate for eps=0.05: 0.805
Success rate for eps=0.06: 0.718
Success rate for eps=0.07: 0.587
Success rate for eps=0.08: 0.46
Success rate for eps=0.09: 0.411
Success rate for eps=0.1: 0.34
